# GMSPY demo

In [1]:
%cd ..
import gmspy
import gams
import pandas as pd
import numpy as np

/home/bill/documents/ecar/fleet-optimization-code


## Define some Data

In [2]:
# 3 sets
cohorts = [str(i) for i in range(2020, 2029)] # Important that this be string, not integers
tech = ['bev', 'icev']
imps = ['gwp']

In [3]:
# total demand - pd Series
Y = pd.Series(index=tech, data=[100, 2])
Y

bev     100
icev      2
dtype: int64

In [4]:
# Stock - 2d DataFrame
STOCK = pd.DataFrame(index=tech, columns=cohorts)
STOCK[:] = np.random.rand(*STOCK.shape)
STOCK

,2020,2021,2022,2023,2024,2025,2026,2027,2028
bev,0.839384,0.606491,0.940869,0.423317,0.652177,0.466471,0.050134,0.824016,0.504331
icev,0.197514,0.947421,0.434148,0.688221,0.147042,0.905718,0.293994,0.941308,0.907836


In [5]:
# IMP - 3d DataFrame
ix = pd.MultiIndex.from_product((tech, imps))
IMP = pd.DataFrame(index=ix, columns=cohorts)
IMP[:] = np.random.rand(*IMP.shape)
IMP

,,2020,2021,2022,2023,2024,2025,2026,2027,2028
bev,gwp,0.619215,0.166964,0.439538,0.137155,0.648548,0.853029,0.408626,0.409136,0.767484
icev,gwp,0.246909,0.490458,0.518501,0.611586,0.958469,0.645308,0.871200,0.394954,0.035056


## Pass to GAMS and save to GDX file

In [6]:
# Define worspace and database
ws = gams.GamsWorkspace()
my_db = ws.add_database()

In [7]:
# Read in sets
g_tech = gmspy.list2set(my_db, tech, 'tech')
g_imps = gmspy.list2set(my_db, imps, 'imps', "All the environmental impacts")
g_cohort = gmspy.list2set(my_db, cohorts, 'cohort', "Year each car was built")

In [8]:
# Read in parameter
g_Y = gmspy.df2param(my_db, df=Y, domains=[g_tech], name='Y', comment="Total Final demand")

In [9]:
# Read in parameter
g_STOCK = gmspy.df2param(my_db, df=STOCK, domains=[g_tech, g_cohort], name='STOCK')

In [10]:
# Read in parameter
g_IMP  = gmspy.df2param(my_db, df=IMP, domains=[g_tech, g_imps, g_cohort], name='IMP')

In [11]:
# Save to gdx file
my_db.export('/tmp/demo.gdx')

## Inspect db or gdx files

In [12]:
# What is in this file?
gmspy.ls(gdx_filepath='/tmp/demo.gdx')

['tech', 'imps', 'cohort', 'Y', 'STOCK', 'IMP']

In [13]:
# What sets are in this my_db?
gmspy.ls(my_db, entity='Set')

['tech', 'imps', 'cohort']

## Extract from db or gdx files

In [14]:
# Extract a set from the database
extracted_set = gmspy.set2list('cohort', my_db)
extracted_set

['2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028']

In [15]:
# Extract a set from the gdx file
another_set = gmspy.set2list('tech', gdx_filepath='/tmp/demo.gdx')
another_set

['bev', 'icev']

In [16]:
# Extract parameter from GDX file
g_IMP_again = gmspy.param2series('IMP', gdx_filepath='/tmp/demo.gdx')
g_IMP_again

bev   gwp  2020    0.619215
           2021    0.166964
           2022    0.439538
           2023    0.137155
           2024    0.648548
           2025    0.853029
           2026    0.408626
           2027    0.409136
           2028    0.767484
icev  gwp  2020    0.246909
           2021    0.490458
           2022    0.518501
           2023    0.611586
           2024    0.958469
           2025    0.645308
           2026    0.871200
           2027    0.394954
           2028    0.035056
dtype: float64

In [18]:
# Use param2df to get it unstacked()
g_IMP_again_df = gmspy.param2df('IMP', gdx_filepath='/tmp/demo.gdx')
g_IMP_again_df

,,2020,2021,2022,2023,2024,2025,2026,2027,2028
bev,gwp,0.619215,0.166964,0.439538,0.137155,0.648548,0.853029,0.408626,0.409136,0.767484
icev,gwp,0.246909,0.490458,0.518501,0.611586,0.958469,0.645308,0.871200,0.394954,0.035056


In [19]:
# 2D dataframe
gmspy.param2df('STOCK', gdx_filepath='/tmp/demo.gdx')

,2020,2021,2022,2023,2024,2025,2026,2027,2028
bev,0.839384,0.606491,0.940869,0.423317,0.652177,0.466471,0.050134,0.824016,0.504331
icev,0.197514,0.947421,0.434148,0.688221,0.147042,0.905718,0.293994,0.941308,0.907836
